In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Define the HybridAutoencoder architecture with LeakyReLU
class HybridAutoencoder(nn.Module):
    def __init__(self):
        super(HybridAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(8, 10),
            nn.LeakyReLU(negative_slope=0.5),
            nn.Linear(10, 8)
        )
        self.decoder = nn.Sequential(
            nn.Linear(8, 10),
            nn.LeakyReLU(negative_slope=0.5),
            nn.Linear(10, 8),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Generate random dataset of 1000 8-bit binary sequences
data = torch.randint(0, 2, (1000, 8)).float()
dataset = TensorDataset(data, data)
dataloader = DataLoader(dataset, batch_size=5, shuffle=True)

# Initialize the model, loss function, and optimizer
model = HybridAutoencoder()
criterion = nn.MSELoss()
optimizer = optim.NAdam(model.parameters(), lr=0.001)

# Train the model
epochs = 100
for epoch in range(epochs):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Save the model parameters
torch.save(model.state_dict(), 'hybrid_autoencoder_parameters.pth')

# Verify the number of trainable parameters
trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total Trainable Parameters: {trainable_parameters}')

# Assuming the statement about 178 parameters per topology is a target to verify against


Epoch 1, Loss: 0.23053038120269775
Epoch 2, Loss: 0.18965253233909607
Epoch 3, Loss: 0.12083496153354645
Epoch 4, Loss: 0.13522782921791077
Epoch 5, Loss: 0.09368416666984558
Epoch 6, Loss: 0.09702236950397491
Epoch 7, Loss: 0.10860507190227509
Epoch 8, Loss: 0.030692970380187035
Epoch 9, Loss: 0.06455062329769135
Epoch 10, Loss: 0.047227878123521805
Epoch 11, Loss: 0.04183494672179222
Epoch 12, Loss: 0.008781780488789082
Epoch 13, Loss: 0.0559607669711113
Epoch 14, Loss: 0.07132793962955475
Epoch 15, Loss: 0.018556276336312294
Epoch 16, Loss: 0.0378124937415123
Epoch 17, Loss: 0.07080216705799103
Epoch 18, Loss: 0.006230912171304226
Epoch 19, Loss: 0.01877507008612156
Epoch 20, Loss: 0.016186052933335304
Epoch 21, Loss: 0.0023237827699631453
Epoch 22, Loss: 0.0059682405553758144
Epoch 23, Loss: 0.03346540406346321
Epoch 24, Loss: 0.0066977208480238914
Epoch 25, Loss: 0.08269567787647247
Epoch 26, Loss: 0.012625059112906456
Epoch 27, Loss: 0.012640422210097313
Epoch 28, Loss: 0.0003586

In [ ]:
# Example 8-bit binary sequence
test_input = torch.tensor([[0, 1, 0, 1, 1, 0, 1, 0]]).float()
print("Test i/p : ", test_input)
# Pass through the encoder only
with torch.no_grad():  # Ensure no gradient is computed to save memory and computations
    encrypted_vector = model.encoder(test_input)
    print('Encrypted vector:', encrypted_vector.numpy())

    decrypted_vector = model.decoder(encrypted_vector)
    print('decrypted vector:', decrypted_vector.numpy())



binarized_output = torch.where(decrypted_vector > 0.5, torch.tensor(1.0), torch.tensor(0.0))

print('Binarized output:', binarized_output)



Test i/p :  tensor([[0., 1., 0., 1., 1., 0., 1., 0.]])
Encrypted vector: [[ 0.02202405  1.4976857  -1.5635588  -2.3500974  -1.4465282   9.93152
  -3.9664261  -3.7735186 ]]
decrypted vector: [[2.7782526e-05 1.0000000e+00 4.5347694e-11 9.9998927e-01 1.0000000e+00
  4.1158108e-17 9.9999964e-01 3.5404420e-07]]
Binarized output: tensor([[0., 1., 0., 1., 1., 0., 1., 0.]])


In [ ]:

correct_predictions = (binarized_output == test_input).float()

# Calculate the accuracy as the mean of correct predictions
accuracy = correct_predictions.mean()

print(f'Accuracy: {accuracy.item() * 100}%')
print(f'Correct predictions: {correct_predictions.sum().item()} out of {test_input.nelement()}')

# You might also want to see the exact comparison
comparison = torch.cat((test_input, binarized_output, correct_predictions), 0)
print('Row 1: Original Input\nRow 2: Binarized Output\nRow 3: Correct Predictions (1: Correct, 0: Incorrect)\n', comparison)


Accuracy: 100.0%
Correct predictions: 8.0 out of 8
Row 1: Original Input
Row 2: Binarized Output
Row 3: Correct Predictions (1: Correct, 0: Incorrect)
 tensor([[0., 1., 0., 1., 1., 0., 1., 0.],
        [0., 1., 0., 1., 1., 0., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])


In [ ]:
# Save the dataset
torch.save(data, 'binary_sequences_dataset.pth')

# Load the dataset
loaded_data = torch.load('binary_sequences_dataset.pth')

# Verify by comparing with original data (optional)
print('Data loaded correctly:', torch.equal(data, loaded_data))



Data loaded correctly: True
